In [ ]:
# import libraries
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns 
import html
from bs4 import BeautifulSoup
import requests
import os
from selenium import webdriver
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import warnings; warnings.simplefilter('ignore')
from VE_scraper_functions import *
from chromedriver_py import binary_path # this will get you the path variable
from selenium.webdriver.common.by import By
import time
import nltk
nltk.download('punkt')

In [ ]:
# import dataset
path = "park_scrape_content_dataset.xlsx"
park_scrape_dataset2 = pd.read_excel(path)
park_scrape_dataset2.head

In [ ]:
## convert all text to lowercase to avoid case sensitivity issues
park_scrape_dataset2['content'] = park_scrape_dataset2['content'].str.lower()
park_scrape_dataset2.head

In [ ]:
EV_words =  [
       "charger", "electric vehicle", " ev ", "charging station"]

In [ ]:
def count_EV_words(text):
    word_counts = {word: text.lower().count(word) for word in EV_words}
    return word_counts
word_counts_df = park_scrape_dataset2['content'].apply(count_EV_words).apply(pd.Series)
word_counts_df['total'] = word_counts_df.sum(axis=1)
word_counts_df.head()

In [ ]:
park_scrape_dataset2 = pd.concat([park_scrape_dataset2, word_counts_df], axis=1)
park_scrape_dataset2 = park_scrape_dataset2.drop("content", axis="columns")
park_scrape_dataset2.head

In [ ]:
park_scrape_dataset2.to_csv("EVtest.csv")

In [ ]:
park_word_counts = park_scrape_dataset2.groupby('park')[EV_words].sum()
park_word_counts['total'] = park_word_counts.sum(axis=1)
park_word_counts.head

In [ ]:
park_word_counts.to_csv("EVbypark.csv")

In [ ]:
path = "clusterlookup.csv"
clusters = pd.read_csv(path)
clusters.head

In [ ]:
clustersubset = clusters[['Park Alpha', "FinalCluster", "Label"]]
clustersubset.rename(columns={'Park Alpha':"park"}, inplace = True)
clusterparks = pd.merge(park_word_counts, clustersubset, on="park", how="left")
clusterparks.head

In [ ]:
filtered_df = clusterparks[clusterparks['total'] > 1]
label_counts = filtered_df.groupby('Label').size().reset_index(name='EV_Parks')
label_counts.head

In [ ]:
label_counts.to_csv("EVbycluster.csv")